In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Step 1: Sample dataset (UserID, MovieID, Rating)
data = {
    'UserID': [1, 1, 2, 2],
    'MovieID': [101, 102, 101, 103],
    'Rating': [5, 3, 4, 2]
}
df = pd.DataFrame(data)

# Step 2: Create user-item matrix
user_item_matrix = df.pivot_table(index='UserID', columns='MovieID', values='Rating').fillna(0)
print("User-Item Matrix:")
print(user_item_matrix)

# Step 3: Compute cosine similarity between users
similarity_matrix = cosine_similarity(user_item_matrix)
similarity_df = pd.DataFrame(similarity_matrix, index=user_item_matrix.index, columns=user_item_matrix.index)
print("\nUser Similarity Matrix:")
print(similarity_df)

# Step 4: Function to get recommendations for a user
def get_recommendations(user_id, user_item_matrix, similarity_df, k=1):
    # Get similar users
    similar_users = similarity_df[user_id].sort_values(ascending=False)[1:k+1].index
    
    # Get movies the target user hasn't rated
    user_ratings = user_item_matrix.loc[user_id]
    unrated_movies = user_ratings[user_ratings == 0].index
    
    # Predict ratings for unrated movies
    predictions = []
    for movie in unrated_movies:
        # Get ratings from similar users for this movie
        similar_user_ratings = user_item_matrix.loc[similar_users, movie]
        # Get similarity scores for those users
        sim_scores = similarity_df.loc[similar_users, user_id]
        # Compute weighted average rating
        if sim_scores.sum() > 0:  # Avoid division by zero
            predicted_rating = np.dot(similar_user_ratings, sim_scores) / sim_scores.sum()
            predictions.append((movie, predicted_rating))
    
    # Sort predictions by predicted rating
    predictions.sort(key=lambda x: x[1], reverse=True)
    return predictions

# Step 5: Get recommendations for User 1
recommendations = get_recommendations(1, user_item_matrix, similarity_df, k=1)
print("\nRecommendations for User 1:")
for movie, score in recommendations:
    print(f"Movie {movie}: Predicted Rating {score:.2f}")

User-Item Matrix:
MovieID  101  102  103
UserID                
1        5.0  3.0  0.0
2        4.0  0.0  2.0

User Similarity Matrix:
UserID         1         2
UserID                    
1       1.000000  0.766965
2       0.766965  1.000000

Recommendations for User 1:
Movie 103: Predicted Rating 2.00
